In [ ]:
# Step 1: Install Required Libraries
!pip install flask rdflib flask-ngrok

# Step 2: Import Necessary Modules
from flask import Flask, render_template_string, request
from rdflib import Graph

# Step 3: Initialize Flask App
app = Flask(__name__)

# Step 4: Load the Ontology
g = Graph()

# Path to the OWL file generated by the previous script
ontology_file = "Updated_ChemistryBasics (1).owl"  # Update if the file is in a different location

try:
    g.parse(ontology_file, format="xml")
    print("Ontology loaded successfully!")
except Exception as e:
    print(f"Error loading ontology: {e}")

# Step 5: Define Function to Query the Ontology
def get_reactions():
    query = """
    SELECT DISTINCT ?reaction ?reactant ?product ?condition
    WHERE {
        ?reaction a :ReactionType ;
                  :hasReactant ?reactant ;
                  :hasProduct ?product ;
                  :requiresCondition ?condition .
    }
    """
    try:
        results = g.query(query, initNs={"": "http://www.semanticweb.org/ashra/ontologies/2024/10/ChemistryBasics#"})
        reactions = {}
        for row in results:
            reaction = row.reaction.split("#")[-1]
            reactant = row.reactant.split("#")[-1]
            product = row.product.split("#")[-1]
            condition = row.condition.split("#")[-1]
            if reaction not in reactions:
                reactions[reaction] = {"reactants": [], "products": [], "conditions": condition}
            reactions[reaction]["reactants"].append(reactant)
            reactions[reaction]["products"].append(product)
        return reactions
    except Exception as e:
        print(f"Error querying ontology: {e}")
        return {}

# Fetch Reaction Data from Ontology
reaction_data = get_reactions()

# Step 6: Define Flask Routes and UI
@app.route("/", methods=["GET", "POST"])
def index():
    selected_reaction = None
    result = ""

    if request.method == "POST":
        selected_reaction = request.form.get("reaction")
        if selected_reaction:
            reactants = ", ".join(reaction_data[selected_reaction]["reactants"])
            products = ", ".join(reaction_data[selected_reaction]["products"])
            conditions = reaction_data[selected_reaction]["conditions"]
            result = f"""
            <h4>Details for {selected_reaction}:</h4>
            <p><b>Reactants:</b> {reactants}</p>
            <p><b>Products:</b> {products}</p>
            <p><b>Conditions:</b> {conditions}</p>
            """

    # HTML Template
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Intelligent Tutoring System</title>
    </head>
    <body style="font-family: Arial, sans-serif; margin: 20px;">
        <h2>Welcome to the Intelligent Tutoring System</h2>
        <form method="POST">
            <label for="reaction">Select a Reaction:</label>
            <select name="reaction" id="reaction" required>
                <option value="">--Choose a reaction--</option>
                {"".join([f"<option value='{reaction}'>{reaction}</option>" for reaction in reaction_data])}
            </select>
            <br><br>
            <button type="submit">Get Details</button>
        </form>
        <hr>
        {result}
    </body>
    </html>
    """
    return render_template_string(html)

# Step 7: Run Flask App
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


Ontology loaded successfully!
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.29.55:5000
Press CTRL+C to quit
192.168.29.55 - - [14/Dec/2024 16:18:16] "GET / HTTP/1.1" 200 -
192.168.29.55 - - [14/Dec/2024 16:18:21] "POST / HTTP/1.1" 200 -
192.168.29.55 - - [14/Dec/2024 16:18:27] "POST / HTTP/1.1" 200 -
